In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
245,AFG,Asia,Afghanistan,2020-09-01,38196.0,34.0,18.000,1406.0,4.0,1.286,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
422,ALB,Europe,Albania,2020-09-01,9513.0,133.0,129.714,284.0,4.0,4.286,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
668,DZA,Africa,Algeria,2020-09-01,44494.0,348.0,376.571,1510.0,9.0,9.143,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-31,France,282380.0,0.0,30632.0,0.0,0.0,0.0
0,2020-08-31,China,89927.0,0.0,4728.0,0.0,0.0,0.0
0,2020-08-31,Italy,269508.0,0.0,35499.0,0.0,0.0,0.0
0,2020-08-31,Spain,440021.0,0.0,29019.0,0.0,0.0,0.0
0,2020-08-31,United States,6084988.0,0.0,184347.0,0.0,0.0,0.0
0,2020-08-31,World,25776265.0,0.0,855910.0,0.0,0.0,0.0
0,2020-08-31,United Kingdom,335643.0,0.0,41545.0,0.0,0.0,0.0
0,2020-08-31,Germany,243189.0,0.0,9309.0,0.0,0.0,0.0
0,2020-08-31,Iran,377827.0,0.0,21721.0,0.0,0.0,0.0
0,2020-08-31,Turkey,269832.0,0.0,6357.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-01,France,282380.0,281025.0,30632.0,30635.0,1355.0,-3.0
0,2020-09-01,China,89927.0,89914.0,4728.0,4723.0,13.0,5.0
0,2020-09-01,Italy,269508.0,269214.0,35499.0,35483.0,294.0,16.0
0,2020-08-31,Spain,440021.0,462858.0,29019.0,29094.0,-22837.0,-75.0
0,2020-09-01,United States,6084988.0,6031013.0,184347.0,183598.0,53975.0,749.0
0,2020-09-01,World,25776265.0,25509135.0,855910.0,850902.0,267130.0,5008.0
0,2020-09-01,United Kingdom,335643.0,335873.0,41545.0,41501.0,-230.0,44.0
0,2020-09-01,Germany,243189.0,243599.0,9309.0,9302.0,-410.0,7.0
0,2020-09-01,Iran,377827.0,375212.0,21721.0,21571.0,2615.0,150.0
0,2020-09-01,Turkey,269832.0,270133.0,6357.0,6370.0,-301.0,-13.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")